# Training your first Convolutional Neural Network (CNN)

In this notebook we will train together your first CNN. Both the problem and the network we will use to solve it are very old: they were introduced in the late 1980s at the very start of deep learning litterature. The dataset is composed of handwritten digits which we will identify as 0s, 1s, 2, etc... It is called MNIST. Nowadays, this task is considered too simple to truly evaluate the power of a deep learning model. Even though the deep learning community still uses this dataset, performing well on MNIST is considered a proof of concept, or even a sanity check, rather than really an achievement. Still, it will make for a good sandbox to present CNNs, their architectures, their building blocks and good practices (and of course a lot of technical _jargon_).

Here is the paper introducing the dataset and the network:
LeCun, Yann and Boser, Bernhard E and Denker, John S and Henderson, Donnie and Howard, Richard E and Hubbard, Wayne E and Jackel, Lawrence D, _Handwritten digit recognition with a back-propagation network_ in Advances in neural information processing systems 1990, pages 396--404.
http://papers.nips.cc/paper/293-handwritten-digit-recognition-with-a-back-propagation-network.pdf

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# auto differentiation library
import torch
from torch.autograd import grad, Function
import torch.nn.functional as F  # usual functions
import torch.nn as nn  # for defining Neural Networks
import torch.optim as optim
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# notebook specific library for displays
from tqdm.notebook import tqdm
from IPython.display import display, Markdown

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"All further computation will be done on device '{device}'")

# A Dataset: Train and Test sets

A dataset is always in two parts: a Train set and a Test set. The reason is that we will optimize our model to fit the training data. Doing so, the model may fit the data __too much__, in the sense that it will not be representative of the true data distribution. Because of this, we can not evaluate the model's performance on the data used to train it: they are not independant.

In order to avoid this problem, we split the dataset into two subsets: a train set, and a test set. We will train the model on the train set while leaving the test set untouched. Then, the model and test samples are still independant so we can use the test set to evaluate the model's performance.

# MNIST

Always start by identifying the task: you should look at your samples and not rush into training your network. This means looking at the raw images, doing some plots and some statistics. This is an important part of the process: here the dataset is already preprocessed and ready to use but often you will come across dataset with problems such as:
- unbalanced classes: e.g. when identifying whales on satellite photos to track whale populations -- most photos will not contain a whale, meaning your dataset will naturally have a bias toward answering no whale is in the picture if you do nothing.
- unrenormalized samples: renormalization is an important part this process, often we will want to train data with zero mean and variance 1 (or at least constant mean and variance, or values between 0 and 1, etc...) as it simplifies the training process a lot.
- wrong labels in your data, or misleading samples: we will come across some difficult samples even in MNIST (1s and 7s can easily look alike). Sometimes, some labels in your dataset can simply be wrong !

Doing this part will also help you to understand how to handle this specific data.

Pytorch already contains the MNIST dataset so we can simply load it:

In [ ]:
# dataset will be stored in a `mnist` directory, pytorch automatically download it if the directory doesn't exist

# Standard transforms used to load the MNIST Dataset: load it as tensors and renormalize
transform = transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize((0.5,), (0.5))])

# Train Set
train_set = MNIST("./mnist/", train=True, download=True, transform=transform)
# Test Set
test_set = MNIST("./mnist/", train=False, download=True, transform=transform)

# Each set is a list-like object containing an image and its corresponding label: a digit 0-9
print(type(train_set[0][0]), type(train_set[0][1]))
print(type(test_set[0][0]), type(test_set[0][1]))

### Make sure that the dataset is balanced, and that the train and test sets are sampled from the same distribution.

In [ ]:
def count_samples(dataset):
    num_samples = len(dataset)
    num_samples_per_label = np.zeros(10)  # 10 different digits
    for __, digit in dataset:  # iterate over all samples in dataset
        num_samples_per_label[digit] += 1  # add 1 to the count for label `digit`
    return num_samples, num_samples_per_label

count, count_detail = count_samples(train_set)
print(f"Training set:  {count} samples with size {train_set[0][0].shape}")
for digit, digit_count in enumerate(count_detail):
    print(f"\tlabel {digit}: {int(digit_count)} samples ( {np.round(100 * digit_count / count, 1)}% )")

print("")

count, count_detail = count_samples(test_set)
print(f"Training set:  {count} samples with size {test_set[0][0].shape}")
for digit, digit_count in enumerate(count_detail):
    print(f"\tlabel {digit}: {int(digit_count)} samples ( {np.round(100 * digit_count / count, 1)}% )")


### For each label, let's find a few samples and show look at them:
Note the variability in each class, and similarity between some 7s and 1s

In [ ]:
def show_samples_with_label(digit, num_samples=5):
    # first we must find `num_samples` different samples with label `digit`
    list_of_samples = []
    num_samples_found = 0
    for sample, label in train_set:
        # check if the sample has the correct label
        if label == digit:  # if so, save it and update count of found samples
            list_of_samples.append(sample)
            num_samples_found += 1
        
            if num_samples_found == num_samples:  # we have found enough samples
                break  # exit the closest while or for loop
    
    # now we need to plot the different samples
    plt.figure(figsize=(12, 4))
    
    for i, sample in enumerate(list_of_samples):
        plt.subplot(1, num_samples, i+1)
        plt.imshow(sample.numpy()[0], cmap='gray')
        # remove tick marks on x and y axis
        plt.xticks([])
        plt.yticks([])
        
    plt.show()
    

for digit in range(10):
    show_samples_with_label(digit)

### Going further
Ideally, we would like to continue analysing the dataset. We could for example plot for each label the histogram of average sample color to see if this simple operation is sufficient to classify digits, or to at least provide significant information (note that this simple operation could potentially expose racial bias in a dataset, so things like that are not to be overlooked).

However, detailled dataset analysis in beyond the scope of this notebook and has already been done extensively on this particular dataset.

# Defining a Network

We will use the CNN called LeNet-5 introduced in the same paper as MNIST (see reference in the first cell of the notebook). Its architecture can be synthetized with the following image:
![title](img/lenet-5.png)

### CNN building blocks
The image describes the operation between each layer of the neural network. These operation are standard, they are building blocks of CNNs, so we only give their name (__convolution__, __subsampling__, __fully connected__) and the number of __channels__.

### What about the non-linearity ?
It is very standard for CNNs to use the ReLU (Rectified Linear Unit):
$$
    ReLU: x \in \mathbb R \mapsto \max(x, 0)
$$
So standard in fact that we often don't mention it, it is the default non-linearity.

Note however that the last layer is called _Gaussian connections_, this is an early-deep-learning name for using a fully connected layer with a soft-max non-linearity:
$$
    Softmax: x = (x_1, \dots, x_D) \in \mathbb R^D \mapsto \left( \frac{e^{x_1}}{\sum_d e^{x_d}}, \dots, \frac{e^{x_D}}{\sum_d e^{x_d}} \right) 
$$
This operation is called softmax because the sum in the denominator is approximately equal to exponential of the maximum value $x_i^{max}$, so $Softmax(x)$ is approximately $1$ at the highest coordinate, and $0$ elsewhere. The "soft" part of the name comes from the fact that the operation is relaxed in order to be differentiable.

#### Motivation for Softmax and link with the name "Gaussian connections":
If $D=2$, this softmax becomes a sigmoid. In fact, the softmax emerges from a Gaussian mixture model of the data. We can think of the whole network in two parts:
- the first part composed of all layers but the last: its role is to find a representation of the data which is linearly separable. The output of this first part is often refered to as the __embedding__.
- the last layer: linear classifier

Using a softmax non-linearity on the last layer corresponds to doing a logistic regression for classifing the samples' embeddings.

### Defining the network in Pytorch
In pytorch, a neural network is defined as a `nn.Module`. This will automatically register all trainable parameters and allow us to use the auto-differention mechanics.
To do so, we define a new class for our network and make it _inherit_ from `nn.Module`. Two methods must be declared:
- `__init__`: which will run at the initialization of the network -- we define there the different layers of the network. Note that since these are standard building blocks, Pytorch has classes for Convolutional, Fully Connected, and lots of other layers already implemented. When calling our class, an instance will be initialized: `__init__` will be called, which will in turn initialize each layer. During each layer's initialization, parameters will be randomly sampled as specified by Pytorch's implementation of these layers.
- `forward`: for objects inheriting from `nn.Module`, this is the method called when using the network as an instance (e.g. if `net` is an instance of our class, `net(x)` will in fact call `net.forward(x)`). There, we apply all layers of the network to the data.


Note: The following code was imported from the Pytorch tutorials: https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#sphx-glr-beginner-blitz-neural-networks-tutorial-py

In [ ]:
class LeNet(nn.Module):

    def __init__(self):
        super().__init__()  # call initialization of the class we are inheriting from: here nn.Module
        # here we only define operations which must be initialized: operations with parameters
        # or that are specific to the size of the input. This is not the case of subsamplings.
        
        # convolution kernels
        self.conv1 = nn.Conv2d(1, 6, 3, padding=2)  # 1 input image channel, 6 output channels, 3x3 square convolution
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        # note that we do not apply the softmax
        # this is handled in the loss to avoid a numerical instability
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
lenet = LeNet()
print(lenet)

# move to GPU if availabe
lenet = lenet.to(device)

# Let's train it !

### First we need to organize batches of data: we can do this easily with Pytorch's DataLoader class

In [ ]:
batch_size = 32
kwargs = {'num_workers': 16, 'pin_memory': (device == 'cuda')}  # these are arguments you don't need to
# understand for now, they are used for speeding up the loading process. If you are interested, you can read
# their definition in the documentation of DataLoader and/or ask me

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, **kwargs)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, **kwargs)

### We must define the Loss, i.e. the quantity that we want to minimize

In [ ]:
criterion = nn.CrossEntropyLoss().to(device)

### We need to define an optimization strategy: an optimizer which encapsulate the gradient descent startegy, and the number of epochs (loop over the whole training set).
Its first argument is the list of parameters that must be trained, which is simply obtained with `lenet.parameters()`.

In [ ]:
optimizer = optim.SGD(lenet.parameters(), lr=1e-2)
n_epoch = 5

### Exercise 1: Define a function `train_one_epoch()` that loops over all batches in the training set, taking a gradient step at each batch.

Note: Don't forget to call `optimizer.zero_grad()` to set gradients to zero between steps !

In [ ]:
run -i solutions/exo1

### Exercise 2: Define a function `evaluate(dataloader)` that returns the network's loss and accuracy on a whole dataset.

In [ ]:
run -i solutions/exo2

### Let's train the network:

In [ ]:
def train_network():
    # define accumulators for loss and accuracy
    train_loss = np.zeros(n_epoch)
    train_accuracy = np.zeros(n_epoch)
    test_loss = np.zeros(n_epoch)
    test_accuracy = np.zeros(n_epoch)

    # tqdm is just showing a progress bar of the iteration over range(n_epoch)
    for epoch in tqdm(range(n_epoch), desc="Training"):
        train_one_epoch()

        tr_loss, tr_acc = evaluate(train_loader)
        train_loss[epoch] = tr_loss
        train_accuracy[epoch] = tr_acc

        te_loss, te_acc = evaluate(test_loader)
        test_loss[epoch] = te_loss
        test_accuracy[epoch] = te_acc

        tqdm.write(
            "Epoch {}: Train loss {:.2E}  accuracy {:.2f}  --  Test loss {:.2E}  accuracy {:.2f}".format(
                epoch, tr_loss, 100*tr_acc, te_loss, 100*te_acc
            )
        )
    return train_loss, train_accuracy, test_loss, test_accuracy


train_loss, train_accuracy, test_loss, test_accuracy = train_network()

### Exercise 3: plot the evolution of loss and accuracy over epochs. Any comments ?

In [ ]:
run -i solutions/exo3

### Exercise 4: play with number of epochs, learning rate and optimizer to train the network and get a good accuracy

In [ ]:
# first we reset everything: the loaders and the network
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, **kwargs)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, **kwargs)
lenet = LeNet().to(device)

Here we redefine the optimizer and number of epochs

In [ ]:
run -i solutions/exo4

In [ ]:
# then we just train the network and plot the results as we did with `train_network` and Exercise 3
train_loss, train_accuracy, test_loss, test_accuracy = train_network()

In [ ]:
run -i solutions/exo3

Note that in this second optimization setup, the training and testing metrics are very different. The difference increases over time. The gap between training and testing is overfitting. Here it is not too harmful: the accuracy on the test set does not decrease. However, with a more complex network the test accuracy may decrease at the end, suggesting that we would beneficiate from stopping the training after fewer epochs.

Note: measuring when the training should stop on the test set is cheating, since the training then depends on the test set. This hyper parameter should thus be set using cross-validation on the training set alone. 